In [39]:
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import *
import torch as T
import torch.nn as nn
import random
import os
project_path= os.path.dirname(os.path.abspath(os.curdir))
import sys
sys.path.insert(0, project_path+ '/Tools')
sys.path.insert(1, project_path+ '/Optimal Control Methods/Learning Methods/Model Free RL')
sys.path.insert(2, project_path+ '/Systems')
from EnforceTyping import enforce_method_typing
from ParticlesandFields import Field, ClassicalParticle, ParticleInField
from DDPG import DDPGAgent, DDPGAlgorithm
T.Tensor.ndim = property(lambda self: len(self.shape))

In [40]:
coulomb_constant = 8.9875e9  # N*m^2/C^2
@dataclass(kw_only=True)
class ElectrostaticField2D(Field):
  """
  A class used to represent a 2D Electrostatic Field

  Attributes
  ----------
  field_sources: dict
      a formatted string to print out what the animal says
  dimensionality: tuple
      a tuple of the dimensionality of the field  

  Methods
  -------
  dynamics(self, observation_position: np.ndarray, time: float) -> np.ndarray:
      Represents the value of the field at any given point(s) or time. 
  potential(self, observation_position: np.ndarray, time: float) -> float:
      Represents the potential due to the field at a given position and/or time  
  potential_difference(self, initial_position: np.ndarray, final_position: np.ndarray, time: float) -> float:
      Represents the potential difference between two positions at a given time in the vector field   
  gradient(self, observation_position: np.ndarray, time: float) -> float:
      Represents the gradient at a given position and/or time in the vector field 
  curl(self, observation_position: np.ndarray, time: float) -> float:
      Represents the curl at a given position and/or time in the vector field 
  divergence(self, observation_position: np.ndarray, time: float) -> float:
      Represents the divergence at a given position and/or time in the vector field
  """
  field_sources: dict
  dimensionality: tuple = (2,)

  def __call__(self, observation_position: np.ndarray) -> np.ndarray:
      return self.dynamics(observation_position)

  def __post_init__(self):
    assert len(self.field_sources["Particle"]) == len(self.field_sources["Position"]), "The length of particles and fields don't match"
    for field_source, _ in zip(self.field_sources["Particle"], self.field_sources["Position"]):
        assert isinstance(field_source, ClassicalParticle),  "The field source is not a particle" 

  @enforce_method_typing
  def dynamics(self, observation_position: np.ndarray) -> np.ndarray:
      """
      This function outputs the field strength due to field sources experienced at any given point(s) or time. 
      This determines the physics of the field (a 2D Electricstatic Field in this case)

      Args:
          observation_position (np.ndarray): The position.

      Returns:
          np.ndarray: The electric field strength vector at the given position.
      """
      electric_field_vector = np.zeros_like(observation_position)
      for field_source, source_position in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          position_vectors = np.broadcast_to(source_position, reversed(observation_position.shape)).T
          displacement_vectors = observation_position - position_vectors
          displacement_magnitude = np.linalg.norm(displacement_vectors, axis=0)
          electric_field_vector += (displacement_vectors * field_source.charge) / displacement_magnitude**3
      electric_field_vector = coulomb_constant * electric_field_vector
      return np.round(electric_field_vector, 3)  # N/C or V/m

  @enforce_method_typing
  def potential(self, observation_position: np.ndarray) -> float:
      """
      Calculate the potential (voltage) at a position in the field.

      Args:
          observation_position (np.ndarray): The position.

      Returns:
          np.ndarray: The electric potential at the given position.
      """
      electric_potential = 0.0
      for field_source, source_position in zip(self.field_sources["Particle"], self.field_sources["Position"]):
          position_vectors = np.broadcast_to(source_position, reversed(observation_position.shape)).T
          displacement_vectors = observation_position - position_vectors
          displacement_magnitude = np.linalg.norm(displacement_vectors, axis=0)
          electric_potential += field_source.charge / displacement_magnitude
      electric_potential = coulomb_constant * electric_potential
      return np.round(electric_potential, 3)  # V

  @enforce_method_typing
  def potential_difference(self, initial_position: np.ndarray, final_position: np.ndarray) -> float:
    """
    Calculate the potential difference between the initial position and the final position in the field.

    Args:
        initial_position (np.ndarray): The starting position.
        final_position (np.ndarray): The ending position.
        resolution (int, optional): The number of intervals to divide the path into. Defaults to 5000.

    Returns:
        float: The work required to move from the initial position to the final position.
    """
    assert initial_position.shape == self.dimensionality, "initial_position has the wrong dimensions"
    assert final_position.shape == self.dimensionality, "final_position has the wrong dimensions"
    PorentialDifference= self.potential(initial_position)- self.potential(final_position)
    return PorentialDifference

  @enforce_method_typing
  def gradient(self, observation_position: np.ndarray, delta: float= 0.001)->np.ndarray:
    """
    This function returns the derivative of the field at a given point

    Args:
        observation_position (np.ndarray): The position.
        delta (float, optional): The step size. Defaults to 0.001.

    Returns: 
      np.ndarray: The gradient of the field at the given position.
    """
    gradient= np.zeros_like(observation_position)
    for i in range(len(observation_position)):
      di= np.zeros_like(observation_position)
      di[i, ] = di[i, ]+delta
      plusdi= observation_position+ di
      minusdi= observation_position- di
      gradient[i]= (self.dynamics(plusdi)- self.dynamics(minusdi))[i]/ (2* delta)
    return gradient

  @enforce_method_typing
  def plot_field(self, low_bound= -5, high_bound= 5, n_vectors= 50):
    """
    This function plots the 2D electric vector field

    Args:
    low_bound (float, optional): The lower bound of the plot. Defaults to -5.
    high_bound (float, optional): The upper bound of the plot. Defaults to 5.
    n_vectors (int, optional): The number of vectors to plot. Defaults to 50.

    """
    observation_position= np.meshgrid(np.linspace(low_bound, high_bound, n_vectors), 
                                    np.linspace(low_bound, high_bound, n_vectors))
    observation_position= np.stack(observation_position)
    xd, yd = self.dynamics(observation_position)
    xd = xd / np.sqrt(xd**2 + yd**2)
    yd = yd / np.sqrt(xd**2 + yd**2)
    color_aara = np.sqrt(xd**2+ yd**2)
    fig, ax = plt.subplots(1,1)
    cp = ax.quiver(observation_position[0],observation_position[1],xd,yd,color_aara)
    fig.colorbar(cp)
    plt.rcParams['figure.dpi'] = 150
    plt.show()

In [41]:
negative_charge= ClassicalParticle(mass=1e-14, charge= -1e-9)
positive_charge= ClassicalParticle(mass=1e-14, charge= 1e-9)
sources = {"Particle": [negative_charge, positive_charge],
           "Position": [np.array([1.0, 1.0]), np.array([-1.0, 1.0])]} 
target= np.array([0.0, 0.0])
test_electric_field= ElectrostaticField2D(field_sources=sources)
point_charge_in_electric_field= ParticleInField(field=test_electric_field, 
                                               particle=positive_charge, 
                                               target=target)

In [42]:
test_actor_layer= (10, 5)
test_critic_layer= (10, 5)
test_actor_activations= (nn.ReLU(), nn.ReLU())
test_critic_activations= (nn.ReLU(), nn.ReLU())
test_agent = DDPGAgent(point_charge_in_electric_field,
                       actor_layers=test_actor_layer, 
                       critic_layers=test_critic_layer,
                       actor_activations= test_actor_activations,
                       critic_activations= test_critic_activations,
                       observation_size= 4,
                       action_size= 2,
                       actor_learning_rate= 0.01,
                       critic_learning_rate= 0.01,
                       soft_update_rate=0.01,
                       max_size=5)

In [43]:
test_state= test_agent.observe()
test_action= test_agent.act(test_state)
test_value= test_agent.critic.forward(test_state, test_action)
# print(test_agent.policy)
print(test_state)
print(test_action)
print(test_value)

tensor([-9.5877, -1.3812,  0.0000,  0.0000])
tensor([ 1.5245e-12, -2.4199e-12])
tensor([-0.1735], grad_fn=<ViewBackward0>)


In [44]:
print(test_agent.memory)
for _ in range(8):
    rand_state= point_charge_in_electric_field.random_state()
    rand_observation= test_agent.observe(rand_state)
    rand_action= test_agent.act(rand_observation)
    rand_next_state, rand_reward, rand_terminal_signal= point_charge_in_electric_field.transition_step(rand_state, np.array(rand_action), test_agent.control_interval) 
    test_agent.memory.append((rand_observation, rand_action, test_agent.observe(rand_next_state), rand_reward, rand_terminal_signal))
    print(test_agent.memory[-1])

deque([], maxlen=5)
(tensor([ 6.9323, -3.5784,  0.0000,  0.0000]), tensor([ 4.5543e-12, -1.0386e-11]), tensor([  6.6715,  -3.2797, -41.1543,  46.9327]), -999.6326961548343, True)
(tensor([-7.0105,  0.2341,  0.0000,  0.0000]), tensor([ 1.3862e-12, -3.0780e-12]), tensor([ -7.3122,   0.1725, -76.3536, -15.6811]), -1000.2998342113375, True)
(tensor([-0.4592, -6.3109,  0.0000,  0.0000]), tensor([-2.1014e-12, -1.2682e-12]), tensor([ 2.9044e-02, -6.4039e+00,  6.3720e+01, -9.9566e+00]), -1000.0762913161367, True)
(tensor([-5.2405,  4.0761,  0.0000,  0.0000]), tensor([ 1.2647e-12, -6.5858e-12]), tensor([ -5.3424,   4.1850, -43.5753,  46.6759]), -1000.1473158503155, True)
(tensor([-0.8473,  8.4306,  0.0000,  0.0000]), tensor([ 5.1967e-13, -1.1839e-11]), tensor([-4.4974e-01,  8.4410e+00,  5.7563e+01, -1.0365e-02]), -999.9799278894652, True)
(tensor([-1.9693,  7.6907,  0.0000,  0.0000]), tensor([ 2.9328e-13, -1.2194e-11]), tensor([-1.5286,  7.9844, 60.2255, 36.9342]), -1000.1906050446163, True)
(t

In [ ]:
batch = random.sample(test_agent.memory, test_agent.batch_size)
observations, actions, next_observations, rewards, dones = zip(*batch)

state = T.stack(observations).to(test_agent.critic.device)
action = T.stack(actions).to(test_agent.critic.device)
reward = T.tensor(rewards, dtype=T.float).unsqueeze(1).to(test_agent.critic.device)
new_state = T.stack(next_observations).to(test_agent.critic.device)
done = T.tensor(dones, dtype=T.float).unsqueeze(1).to(test_agent.critic.device)

test_agent.target_actor.eval()
test_agent.target_critic.eval()
test_agent.critic.eval()

target_actions = self.actor.forward(new_state)
critic_value_ = self.target_critic.forward(new_state, target_actions) 
q_expected = self.critic.forward(state, action)
q_targets = reward + self.discount_rate * critic_value_ * (1 - done)

critic_loss = nn.MSELoss()(q_expected, q_targets.detach())
self.critic.train()
self.critic.optimizer.zero_grad()
critic_loss.backward()
self.critic.optimizer.step()

self.actor.eval()
self.critic.eval()

mu = self.actor.forward(state)
Actor_loss = -self.critic.forward(state, mu)

Actor_loss = T.mean(Actor_loss)
self.actor.train()
self.actor.optimizer.zero_grad()
Actor_loss.backward()
self.actor.optimizer.step()

self.update_network_parameters()

In [ ]:
test_trajectory, test_trajectory_return= test_agent.sample_trajectory(10.0)
test_agent.plot_trajectory(test_trajectory)

In [ ]:
DDPGAlgorithm(point_charge_in_electric_field, test_agent, 800, 15)

In [ ]:
path, path_return= test_agent.sample_trajectory(10.0)
test_agent.plot_trajectory(path)